## Face Detection

In [1]:
import cv2 as cv
import time
import mediapipe as mp

In [ ]:
cap = cv.VideoCapture(0)
pTime = 0

mpFaceDetection = mp.solutions.face_detection
mpDraw = mp.solutions.drawing_utils
faceDetection = mpFaceDetection.FaceDetection(0.75)

while True:
    success , img = cap.read()
    imgRGB = cv.cvtColor(img,cv.COLOR_BGR2RGB)
    results = faceDetection.process(imgRGB)

    # Detecting Landmarks
    if results.detections:
        for id ,detection in enumerate(results.detections):
            print("ID : ",id , "Detections : ",detection)
            print("Detection Score : ",detection.score)
            print("Bounding Box : ",detection.location_data.relative_bounding_box)
            mpDraw.draw_detection(img,detection)
            
            # Drawing
            h,w,c = img.shape
            bboxC = detection.location_data.relative_bounding_box # Bounding Box Class
            bbox = int(bboxC.xmin * w) , int(bboxC.ymin * h) , int(bboxC.width * w) , int(bboxC.height * h)    
            
            cv.rectangle(img , bbox , (255,0,255) , 2)
            cv.putText(img,f"Score : {int(detection.score[0]*100)}%" , (bbox[0],bbox[1]-20),cv.FONT_HERSHEY_PLAIN , 2 , (255,0,255),2)
    
    # Creating FBS
    cTime = time.time()
    fbs = 1 / (cTime - pTime)
    pTime = cTime
    cv.putText(img,f"FBS : {int(fbs)}" , (20,70),cv.FONT_HERSHEY_PLAIN , 3 , (0,255,0))
    
    # Showing Image
    cv.imshow("Image" , img)
    cv.waitKey(1)

## Module

In [30]:
class FaceDetector():
    def __init__(self, minDetectionConf = 0.5 ):
        self.minDetectionConf=minDetectionConf

        self.mpFaceDetection = mp.solutions.face_detection
        self.mpDraw = mp.solutions.drawing_utils
        self.faceDetection = mpFaceDetection.FaceDetection(self.minDetectionConf)
        
    def findFaces(self,img , draw = True):
        imgRGB = cv.cvtColor(img,cv.COLOR_BGR2RGB)
        self.results = self.faceDetection.process(imgRGB)

        # Detecting Landmarks
        bboxs = [] # Bounding Boxes
        if self.results.detections:
            for id ,detection in enumerate(self.results.detections):
                print("ID : ",id , "Detections : ",detection)
                print("Detection Score : ",detection.score)
                print("Bounding Box : ",detection.location_data.relative_bounding_box)
                mpDraw.draw_detection(img,detection)
                
                # Drawing
                h,w,c = img.shape
                bboxC = detection.location_data.relative_bounding_box # Bounding Box Class
                bbox = int(bboxC.xmin * w) , int(bboxC.ymin * h) , int(bboxC.width * w) , int(bboxC.height * h)    
                # bboxs.append([id ,bbox , detection.score])
                
                # cv.rectangle(img , bbox , (255,0,255) , 2)
                if draw : img = self.fancyDraw(img,bbox)
                cv.putText(img,f"Score : {int(detection.score[0]*100)}%" , (bbox[0],bbox[1]-20),cv.FONT_HERSHEY_PLAIN , 2 , (255,0,255),2)
                
        return img,bboxs
    
    def fancyDraw(self,img, bbox , l = 30):
        x,y,w,h = bbox
        x1 , y1 = x+w , y+h
        
        cv.rectangle(img , bbox , (255,0,255) , 1)
        cv.line(img,(x,y) , (x+l,y) , (255,0,255) , 10) # Top Left Horizonal Line
        cv.line(img,(x,y) , (x,y+l) , (255,0,255) , 10) # Top Left Vertical Line
        
        cv.line(img,(x1,y1) , (x1-l,y1) , (255,0,255) , 10) # Botom Right Horizonal Line
        cv.line(img,(x1,y1) , (x1,y1-l) , (255,0,255) , 10) # Botom Right Vertical Line
        
        return img
        

In [ ]:
cap = cv.VideoCapture(0)
pTime = 0
detector = FaceDetector()

while True:
    success , img = cap.read()
    img,bboxs = detector.findFaces(img)
    
    # Creating FBS
    cTime = time.time()
    fbs = 1 / (cTime - pTime)
    pTime = cTime
    
    cv.putText(img,f"FBS : {int(fbs)}" , (20,70),cv.FONT_HERSHEY_PLAIN , 3 , (0,255,0),2)
    
    # Showing Image
    cv.imshow("Image" , img)
    cv.waitKey(1)